In [1]:
#inisialisasi library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from yellowbrick.classifier import ClassificationReport

In [2]:
#siapkan dataframe untuk mengambil attribut
df = pd.read_csv("\\Users\\aftermath\\Documents\\Machine Learning\\logisticregression\\pd-speech-features.csv",header=None, skipinitialspace=True)

In [6]:
#masukan attribut kedalam array 
attrs = []
for attr in range(754):
    attrs.append(df.at[1,attr])

attrs.append('target')
attrs

['id',
 'gender',
 'PPE',
 'DFA',
 'RPDE',
 'numPulses',
 'numPeriodsPulses',
 'meanPeriodPulses',
 'stdDevPeriodPulses',
 'locPctJitter',
 'locAbsJitter',
 'rapJitter',
 'ppq5Jitter',
 'ddpJitter',
 'locShimmer',
 'locDbShimmer',
 'apq3Shimmer',
 'apq5Shimmer',
 'apq11Shimmer',
 'ddaShimmer',
 'meanAutoCorrHarmonicity',
 'meanNoiseToHarmHarmonicity',
 'meanHarmToNoiseHarmonicity',
 'minIntensity',
 'maxIntensity',
 'meanIntensity',
 'f1',
 'f2',
 'f3',
 'f4',
 'b1',
 'b2',
 'b3',
 'b4',
 'GQ_prc5_95',
 'GQ_std_cycle_open',
 'GQ_std_cycle_closed',
 'GNE_mean',
 'GNE_std',
 'GNE_SNR_TKEO',
 'GNE_SNR_SEO',
 'GNE_NSR_TKEO',
 'GNE_NSR_SEO',
 'VFER_mean',
 'VFER_std',
 'VFER_entropy',
 'VFER_SNR_TKEO',
 'VFER_SNR_SEO',
 'VFER_NSR_TKEO',
 'VFER_NSR_SEO',
 'IMF_SNR_SEO',
 'IMF_SNR_TKEO',
 'IMF_SNR_entropy',
 'IMF_NSR_SEO',
 'IMF_NSR_TKEO',
 'IMF_NSR_entropy',
 'mean_Log_energy',
 'mean_MFCC_0th_coef',
 'mean_MFCC_1st_coef',
 'mean_MFCC_2nd_coef',
 'mean_MFCC_3rd_coef',
 'mean_MFCC_4th_coef',


In [4]:
#dataframe baru dengan nama kolom = attrs
dfnew = pd.read_csv("\\Users\\aftermath\\Documents\\Machine Learning\\logisticregression\\pd-speech-features.csv",header=None)
dfnew = df.iloc[2:]
dfnew.drop(dfnew.index[[0,2]])
dfnew.columns = attrs
dfnew.index =  range(len(dfnew.index))
print(dfnew.info())
print(dfnew.describe())

ValueError: Length mismatch: Expected axis has 755 elements, new values have 753 elements

In [ ]:
#dataframe yang dibentuk
dfnew.head(20)

In [ ]:
#cek apakah ada data yang hilang
missing_values = dfnew.isnull()
missing_values

sns.heatmap(data = missing_values)

In [ ]:
#melihat perbandingan jumlah kelas label 1 dan 0
sns.countplot(x='target', data=dfnew)
dfnew.target.value_counts()

In [ ]:
#prepare train-test data
features = dfnew[feature]
label = dfnew['target']

X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.25, random_state=101)

In [ ]:
logmodel = LogisticRegression()
logmodel.fit(X_train, y_train)

In [ ]:
#evaluate the model
classes = ['parkinson','not-parkinson']
y_pred = logmodel.predict(X_test)
print(classification_report(y_test, y_pred, target_names=classes))

visualizer = ClassificationReport(logmodel, classes=classes, support=True)
visualizer.fit(X_train, y_train)  # Fit the visualizer and the model
visualizer.score(X_test, y_test)  # Evaluate the model on the test data
g = visualizer.poof()             # Draw/show/poof the data